<a href="https://colab.research.google.com/github/flaviohnm/projeto_bancodedados/blob/main/recomendacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Instalação do módulos

In [5]:
pip install pandera

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 197 kB 5.1 MB/s 


#Bloco de Extração

In [6]:
import pandas as pd
import pandera as pa

In [7]:
df = pd.read_csv("/content/drive/MyDrive/CENIPA/recomendacao.csv",sep=";",parse_dates=['recomendacao_numero'],dayfirst=True)

## O comando "head" exibe a quantidade de linhas contida no parentesis.

In [9]:
df.head(10)

,codigo_ocorrencia4,recomendacao_numero,recomendacao_dia_assinatura,recomendacao_dia_encaminhamento,recomendacao_dia_feedback,recomendacao_conteudo,recomendacao_status,recomendacao_destinatario_sigla,recomendacao_destinatario
0,45331,A-582/CENIPA/2014 - 01,29/07/2016,25/08/2016,20/12/2016,Atuar junto às empresas que operam segundo o R...,CUMPRIDA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
1,45331,A-582/CENIPA/2014 - 02,29/07/2016,25/08/2016,20/12/2016,Atuar junto à Administração do Aeroporto Inter...,CUMPRIDA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
2,45331,A-582/CENIPA/2014 - 03,29/07/2016,25/08/2016,NaN,Orientar as suas organizações subordinadas em ...,AGUARDANDO RESPOSTA,DECEA,DEPARTAMENTO DE CONTROLE DE ESPAÇO AÉREO
3,45392,A-032/CENIPA/2014 - RSV 001,07/04/2014,14/04/2015,21/08/2015,Divulgar aos operadores de helicópteros vincul...,CUMPRIDA DE FORMA ALTERNATIVA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
4,45392,A-032/CENIPA/2014 - RSV 002,07/04/2014,14/04/2015,17/08/2015,Tendo em vista os fatos levantados nesta inves...,NÃO CUMPRIDA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
5,45397,IG-536/CENIPA/2014 - 01,15/04/2016,17/05/2016,21/02/2017,"Certificar-se que a IS n° 00-002B, no capítulo...",CUMPRIDA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
6,45397,IG-536/CENIPA/2014 - 02,15/04/2016,17/05/2016,24/02/2017,Certificar-se de que o tempo limite para revis...,CUMPRIDA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
7,45400,IG-544/CENIPA/2015 - 01,31/03/2017,31/03/2017,NaN,Assegurar-se que os pilotos que operam aeronav...,AGUARDANDO RESPOSTA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
8,45400,IG-544/CENIPA/2015 - 02,31/03/2017,31/03/2017,NaN,"Realizar gestões junto ao operador, a fim de a...",AGUARDANDO RESPOSTA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
9,45400,IG-544/CENIPA/2015 - 03,31/03/2017,31/03/2017,NaN,"Realizar gestões junto ao operador, a fim de g...",AGUARDANDO RESPOSTA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL


Exibir os tipos de dados dos objetos

In [10]:
df.dtypes

codigo_ocorrencia4                  int64
recomendacao_numero                object
recomendacao_dia_assinatura        object
recomendacao_dia_encaminhamento    object
recomendacao_dia_feedback          object
recomendacao_conteudo              object
recomendacao_status                object
recomendacao_destinatario_sigla    object
recomendacao_destinatario          object
dtype: object

Até esse trecho foi realizado o carregamento de dados com a conversão das colunas que contém datas para o tipo de dado "date" com o método "parse".

In [11]:
df.replace(['***','**','####','###!','****','*****','NULL'], pd.NA, inplace=True)

In [12]:
schema = pa.DataFrameSchema(
    columns = {
        "codigo_ocorrencia4":pa.Column(pa.Int),
        "recomendacao_numero":pa.Column(pa.String),
        "recomendacao_dia_assinatura":pa.Column((pa.String),nullable=True),
        "recomendacao_dia_encaminhamento":pa.Column((pa.String),nullable=True),
        "recomendacao_dia_feedback":pa.Column((pa.String),nullable=True),
        "recomendacao_conteudo":pa.Column((pa.String),nullable=True),
        "recomendacao_status":pa.Column((pa.String),nullable=True),
        "recomendacao_destinatario_sigla":pa.Column((pa.String),nullable=True),
        "recomendacao_destinatario":pa.Column((pa.String),nullable=True)
    }
)

In [13]:
df.isnull().sum()

codigo_ocorrencia4                   0
recomendacao_numero                  0
recomendacao_dia_assinatura          0
recomendacao_dia_encaminhamento      0
recomendacao_dia_feedback          432
recomendacao_conteudo                2
recomendacao_status                  7
recomendacao_destinatario_sigla      0
recomendacao_destinatario            0
dtype: int64

##Validação do Schema

In [14]:
schema.validate(df)

,codigo_ocorrencia4,recomendacao_numero,recomendacao_dia_assinatura,recomendacao_dia_encaminhamento,recomendacao_dia_feedback,recomendacao_conteudo,recomendacao_status,recomendacao_destinatario_sigla,recomendacao_destinatario
0,45331,A-582/CENIPA/2014 - 01,29/07/2016,25/08/2016,20/12/2016,Atuar junto às empresas que operam segundo o R...,CUMPRIDA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
1,45331,A-582/CENIPA/2014 - 02,29/07/2016,25/08/2016,20/12/2016,Atuar junto à Administração do Aeroporto Inter...,CUMPRIDA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
2,45331,A-582/CENIPA/2014 - 03,29/07/2016,25/08/2016,NaN,Orientar as suas organizações subordinadas em ...,AGUARDANDO RESPOSTA,DECEA,DEPARTAMENTO DE CONTROLE DE ESPAÇO AÉREO
3,45392,A-032/CENIPA/2014 - RSV 001,07/04/2014,14/04/2015,21/08/2015,Divulgar aos operadores de helicópteros vincul...,CUMPRIDA DE FORMA ALTERNATIVA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
4,45392,A-032/CENIPA/2014 - RSV 002,07/04/2014,14/04/2015,17/08/2015,Tendo em vista os fatos levantados nesta inves...,NÃO CUMPRIDA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
...,...,...,...,...,...,...,...,...,...
1192,79713,A-146/CENIPA/2020 - 02,08/07/2021,26/07/2021,NaN,Atuar junto à empresa Plá e Silva Aviação Agrí...,AGUARDANDO RESPOSTA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
1193,79713,A-146/CENIPA/2020 - 03,08/07/2021,26/07/2021,NaN,Divulgar os ensinamentos colhidos na presente ...,AGUARDANDO RESPOSTA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
1194,80073,A-063/CENIPA/2021 - 01,12/11/2021,19/11/2021,NaN,Divulgar os ensinamentos colhidos nesta invest...,AGUARDANDO RESPOSTA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
1195,80073,A-063/CENIPA/2021 - 02,12/11/2021,19/11/2021,NaN,Divulgar os ensinamentos colhidos nesta invest...,AGUARDANDO RESPOSTA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL


##Checando se o valor da coluna "codigo ocorrência" são únicos.

In [15]:
df.codigo_ocorrencia4.is_unique

False

Definindo "codigo da ocorrência" como indice

In [16]:
df.set_index('codigo_ocorrencia4', inplace=True)

Consultar quantos NA tem no dataframe

In [17]:
df.isna().sum()

recomendacao_numero                  0
recomendacao_dia_assinatura          0
recomendacao_dia_encaminhamento      0
recomendacao_dia_feedback          432
recomendacao_conteudo                2
recomendacao_status                  7
recomendacao_destinatario_sigla      0
recomendacao_destinatario            0
dtype: int64

In [18]:
df.isnull().sum()

recomendacao_numero                  0
recomendacao_dia_assinatura          0
recomendacao_dia_encaminhamento      0
recomendacao_dia_feedback          432
recomendacao_conteudo                2
recomendacao_status                  7
recomendacao_destinatario_sigla      0
recomendacao_destinatario            0
dtype: int64